In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import sys
import time
import sklearn
import tensorflow as tf
from tensorflow import keras
import pandas as pd

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd,  sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.1.0
sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)
matplotlib 3.1.3
numpy 1.18.1
pandas 1.0.1
sklearn 0.21.0
tensorflow 2.1.0
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf


In [2]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [3]:
from sklearn.model_selection import train_test_split

x_train_all, x_test, y_train_all, y_test = train_test_split(housing.data, housing.target, random_state =7)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_all, y_train_all, random_state =11)

print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(3870, 8) (3870,)
(11610, 8) (11610,)
(5160, 8) (5160,)


In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [10]:
def customized_mse(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_pred - y_true))

model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu',
                       input_shape = x_train.shape[1:]),
    keras.layers.Dense(1),
])
model.summary()
model.compile(loss = customized_mse,
              optimizer = 'sgd',
              metrics = ['mean_squared_error'])
callbacks = [keras.callbacks.EarlyStopping(
    patience=5, min_delta=1e-2)]

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 30)                270       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 31        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [11]:
history = model.fit(x_train_scaled, y_train,
                    epochs = 100,
                    validation_data = [x_valid_scaled, y_valid],
                    callbacks = callbacks)

Train on 11610 samples, validate on 3870 samples
Epoch 1/100
11610/11610 [==============================] - 0s 40us/sample - loss: 1.1434 - mean_squared_error: 1.1434 - val_loss: 0.8152 - val_mean_squared_error: 0.8152
Epoch 2/100
11610/11610 [==============================] - 0s 30us/sample - loss: 1.1462 - mean_squared_error: 1.1462 - val_loss: 0.6598 - val_mean_squared_error: 0.6598
Epoch 3/100
11610/11610 [==============================] - 0s 29us/sample - loss: 0.4896 - mean_squared_error: 0.4896 - val_loss: 0.4611 - val_mean_squared_error: 0.4611
Epoch 4/100
11610/11610 [==============================] - 0s 28us/sample - loss: 0.4195 - mean_squared_error: 0.4195 - val_loss: 0.4244 - val_mean_squared_error: 0.4244
Epoch 5/100
11610/11610 [==============================] - 0s 32us/sample - loss: 0.3964 - mean_squared_error: 0.3964 - val_loss: 0.4108 - val_mean_squared_error: 0.4108
Epoch 6/100
11610/11610 [==============================] - 0s 29us/sample - loss: 0.3838 - mean_squar

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize = (8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    plt.show()
    
plot_learning_curves(history)